## Container preparation
Notebook creates different containers with N number of physical partitions, to ensure you have different scenarios to execute test scenarios. <br/>
Logical partition key will be a generic '/PartitionKey'.

- Database name: Models <br/>
- Container with 10 physical partitions: Cnt10PhysicalPartitions <br/>
- Container with 5 physical partitions: Cnt5PhysicalPartitions <br/>

In [1]:
from azure.cosmos import CosmosClient, PartitionKey

cosmosAccountURI = os.environ['COSMOS_ACCOUNT_URI']
cosmosAccountKey = os.environ['COSMOS_ACCOUNT_KEY']

databaseName = 'Models'

client = CosmosClient(cosmosAccountURI, cosmosAccountKey)
db = client.create_database_if_not_exists(databaseName)

In [2]:
## Container 10 physical partitions
ctrName10pp = 'Cnt10PhysicalPartitions'
partitionKeypath = '/PartitionKey'

pkPath = PartitionKey(path=partitionKeypath)
ctr10pp = db.create_container_if_not_exists(id=ctrName10pp, partition_key=pkPath, offer_throughput=60000) 

# Manual throughput 60000 => 10 Physical partitions
# https://learn.microsoft.com/en-us/azure/cosmos-db/scaling-provisioned-throughput-best-practices#step-3-calculate-the-number-of-rus-to-start-with

# Lower throughput after creation to avoid high RU charge
ctr10pp.replace_throughput(throughput=1000)

In [3]:
## Container 5 physical partitions
ctrName5pp = 'Cnt5PhysicalPartitions'
partitionKeypath = '/PartitionKey'

pkPath = PartitionKey(path=partitionKeypath)
ctr5pp = db.create_container_if_not_exists(id=ctrName5pp, partition_key=pkPath, offer_throughput=30000) 

# Manual throughput 30000 => 10 Physical partitions
# https://learn.microsoft.com/en-us/azure/cosmos-db/scaling-provisioned-throughput-best-practices#step-3-calculate-the-number-of-rus-to-start-with

# Lower throughput after creation to avoid high RU charge
ctr5pp.replace_throughput(throughput=500)

### Clean up containers
Approach below uses TTL to remove all documents in the container, it will set it to 60 seconds and wait for purge to happen. <br/>
At the end TTL is reverted back to None and RU minimized.

In [4]:
# Purge all documents by setting a container TTL
# You have 1 minute to regret and change it, before documents start to be purged...

db.replace_container(container=ctr10pp, partition_key=pkPath, default_ttl=60) 
db.replace_container(container=ctr5pp, partition_key=pkPath, default_ttl=60) 

<ContainerProxy [dbs/Models/colls/Cnt5PhysicalPartitions]>

In [5]:
from time import sleep

# Iterate while items exists...
items = list(ctr10pp.query_items(query='SELECT TOP 1 c.id, c._ts FROM c ORDER BY c._ts DESC', enable_cross_partition_query=True, max_item_count=10))
hasItems = items.__len__()

while hasItems:
    print('Items still exist in the container, waiting for purge...')
    sleep(30)

    items = list(ctr10pp.query_items(query='SELECT TOP 1 c.id, c._ts FROM c ORDER BY c._ts DESC', enable_cross_partition_query=True, max_item_count=10))
    hasItems = items.__len__()

# print(items.__len__())
# print (float(ctr10pp.client_connection.last_response_headers['x-ms-request-charge']))

print('No more items found for container ' + ctrName10pp)

No more items found for container Cnt10PhysicalPartitions


In [6]:
from time import sleep

# Iterate while items exists...
items = list(ctr5pp.query_items(query='SELECT TOP 1 c.id, c._ts FROM c ORDER BY c._ts DESC', enable_cross_partition_query=True, max_item_count=10))
hasItems = items.__len__()

while hasItems:
    print('Items still exist in the container, waiting for purge...')
    sleep(30)

    items = list(ctr5pp.query_items(query='SELECT TOP 1 c.id, c._ts FROM c ORDER BY c._ts DESC', enable_cross_partition_query=True, max_item_count=10))
    hasItems = items.__len__()

# print(items.__len__())
# print (float(ctr10pp.client_connection.last_response_headers['x-ms-request-charge']))

print('No more items found for container ' + ctrName5pp)

No more items found for container Cnt5PhysicalPartitions


In [12]:
# Disable TTL for containers

db.replace_container(container=ctr10pp, partition_key=pkPath, default_ttl=None) 
db.replace_container(container=ctr5pp, partition_key=pkPath, default_ttl=None) 

ctr10pp.replace_throughput(throughput=600)
ctr5pp.replace_throughput(throughput=400)